# Patch boundary plots

In [ ]:
import yaml
from pathlib import Path


package_root = Path.home() / "skymap-convert"

In [ ]:
from skymap_convert import load_pickle_skymap

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## Util functions

In [ ]:
from skymap_convert.geometry import unit_vector3d_to_radec
from lsst.sphgeom import LonLat

In [ ]:
# Get tract and then patches for a specific tract, from lsst_skymap


def get_orig_tract_patches(tract_id, patch_id):
    """Get original patches for a specific tract ID."""
    tract_data = lsst_skymap.generateTract(tract_id)
    return tract_data.getPatchInfo(patch_id)


def get_orig_patch_radec_verts(tract_id, patch_id):
    return [
        unit_vector3d_to_radec(vert)
        for vert in get_orig_tract_patches(tract_id, patch_id).inner_sky_polygon.getVertices()
    ]


def get_orig_tract_radec_verts(tract_id, inner=True):
    tract_data = lsst_skymap.generateTract(tract_id)
    if not inner:
        uv_verts = tract_data.outer_sky_polygon.getVertices()
        return [unit_vector3d_to_radec(vert) for vert in uv_verts]
    box = tract_data.inner_sky_region
    lon_a, lon_b = box.getLon().getA().asRadians(), box.getLon().getB().asRadians()
    lon_min = min(lon_a, lon_b)
    lon_max = max(lon_a, lon_b)
    lat_a, lat_b = box.getLat().getA().asRadians(), box.getLat().getB().asRadians()
    lat_min = min(lat_a, lat_b)
    lat_max = max(lat_a, lat_b)
    corners = [
        LonLat.fromRadians(lon_min, lat_min),
        LonLat.fromRadians(lon_max, lat_min),
        LonLat.fromRadians(lon_max, lat_max),
        LonLat.fromRadians(lon_min, lat_max),
    ]
    return [(corner.getLon().asDegrees(), corner.getLat().asDegrees()) for corner in corners]

In [ ]:
get_orig_patch_radec_verts(3, 4)

## A tract with patches

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u


def _plot_patches(ax, patches, single_color=None, alpha=0.5, marker_size=5):
    """Plot the patches on the given axes."""
    for patch in patches:
        ra, dec = zip(*patch)
        skycoord = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame="icrs")
        ra_deg = skycoord.ra.wrap_at(360 * u.deg).deg
        dec_deg = skycoord.dec.deg

        # Close the polygon
        ra_deg = np.append(ra_deg, ra_deg[0])
        dec_deg = np.append(dec_deg, dec_deg[0])

        # Plot the patches with a single color if specified
        if single_color is not None:
            ax.plot(
                ra_deg,
                dec_deg,
                color=single_color,
                marker="o",
                linestyle="-",
                alpha=alpha,
                markersize=marker_size,
            )
        else:
            ax.plot(ra_deg, dec_deg, marker="o", linestyle="-", alpha=alpha, markersize=marker_size)


def _plot_tract(ax, tract_id, color, inner=True):
    """Plot the tract boundary for a given tract ID."""
    tract_radec_verts = get_orig_tract_radec_verts(tract_id, inner=inner)
    ra, dec = zip(*tract_radec_verts)
    skycoord = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame="icrs")
    ra_deg = skycoord.ra.wrap_at(360 * u.deg).deg
    dec_deg = skycoord.dec.deg

    # Close the polygon
    ra_deg = np.append(ra_deg, ra_deg[0])
    dec_deg = np.append(dec_deg, dec_deg[0])

    # Plot the tract boundary
    if inner:
        # Plot with a solid color fill
        ax.fill(ra_deg, dec_deg, color=color, alpha=0.25, label=f"Tract {tract_id} Inner")
    else:
        ax.plot(ra_deg, dec_deg, color=color, linewidth=3, alpha=0.8, label=f"Tract {tract_id} Outer")


def _get_ra_dec_range(patches):
    """Get the RA/Dec range from the patches."""
    min_ra, max_ra = float("inf"), float("-inf")
    min_dec, max_dec = float("inf"), float("-inf")
    for patch in patches:
        ra, dec = zip(*patch)
        min_ra = min(min(ra), min_ra)
        max_ra = max(max(ra), max_ra)
        min_dec = min(min(dec), min_dec)
        max_dec = max(max(dec), max_dec)
    return (min_ra, max_ra, min_dec, max_dec)


def plot_patches_in_tract(tract_id):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot the patches and tract boundaries
    patches = [get_orig_patch_radec_verts(tract_id, patch_seq_index) for patch_seq_index in range(100)]
    _plot_patches(ax, patches)
    _plot_tract(ax, tract_id, inner=True, color="black")
    _plot_tract(ax, tract_id, inner=False, color="black")

    # Set zoom level based on the RA/Dec range of the patches
    min_ra, max_ra, min_dec, max_dec = _get_ra_dec_range(patches)
    ax.set_xlim(min_ra - 1, max_ra + 1)
    ax.set_ylim(min_dec - 0.25, max_dec + 0.25)

    # Add a legend for the tract boundaries, labels, and title
    ax.legend(loc="upper right", fontsize="small")
    ax.set_xlabel("RA (deg)")
    ax.set_ylabel("Dec (deg)")
    ax.set_title(f"Zoomed-In View of Patches for Tract {tract_id}")
    ax.grid(True)

    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Tract 9, which is near the south pole

plot_patches_in_tract(9)

In [ ]:
# Tract 9000, which is near the equator

plot_patches_in_tract(9000)

## Tract/patch overlap

In [ ]:
def plot_multiple_tract_patches(tract_ids):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Set colors for each tract
    patch_colors = ["C9", "C8", "C6", "C7", "C4"]
    tract_colors = ["C0", "C2", "C3", "C7", "C4"]

    # Initialize RA/Dec range variables - these will be used to set the zoom level later
    min_ra, max_ra, min_dec, max_dec = float("inf"), float("-inf"), float("inf"), float("-inf")

    # Plot the patches and tract boundaries
    for i, tract_id in enumerate(tract_ids):
        # Get patches for the current tract
        patches = [get_orig_patch_radec_verts(tract_id, patch_seq_index) for patch_seq_index in range(100)]

        # Plot the patches and tract boundaries
        _plot_patches(
            ax, patches, single_color=patch_colors[i % len(patch_colors)], alpha=0.25, marker_size=2
        )
        _plot_tract(ax, tract_id, inner=True, color=tract_colors[i % len(tract_colors)])
        _plot_tract(ax, tract_id, inner=False, color=tract_colors[i % len(tract_colors)])

        # Update the RA/Dec range
        tract_min_ra, tract_max_ra, tract_min_dec, tract_max_dec = _get_ra_dec_range(patches)
        min_ra = min(min_ra, tract_min_ra)
        max_ra = max(max_ra, tract_max_ra)
        min_dec = min(min_dec, tract_min_dec)
        max_dec = max(max_dec, tract_max_dec)

    # Set zoom level based on the RA/Dec range of the patches
    ax.set_xlim(min_ra - 1, max_ra + 1)
    ax.set_ylim(min_dec - 0.25, max_dec + 0.25)

    # Add a legend for the tract boundaries, labels, and title
    ax.set_xlabel("RA (deg)")
    ax.set_ylabel("Dec (deg)")
    ax.set_title(f'Zoomed-In View of Patches for Tracts {", ".join(map(str, tract_ids))}')
    ax.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_multiple_tract_patches([7, 8, 9])

In [ ]:
plot_multiple_tract_patches([9000, 9001, 9002])

In [ ]:
plot_multiple_tract_patches([8, 9, 22, 23, 24])  # , 43, 44